In [4]:
from IPython.display import display, clear_output
import utils
import ipywidgets as widgets
from app_widgets import AppWidgets

In [ ]:
# Widgets
config = utils.read_json("config.json")
sample_preparation_options = [object_key for object_key, object_info in config["objects"].items() if object_info["object_type"] == "sample_preparation"]
sample_preparation_dropdown = utils.Dropdown(description='Sample Preparation Task', 
                                           disabled=False, 
                                           options = sample_preparation_options,
                                           value = sample_preparation_options[0],
                                           layout = widgets.Layout(width = '400px'),
                                           style = {"description_width": '145px'})

In [ ]:
def sample_preparation_widgets(change):
    
    clear_output()
    
    display(sample_preparation_dropdown)
    
    task = sample_preparation_dropdown.value
    
    app_widgets = AppWidgets(task, "config.json")

    # Populate dropdown lists
    app_widgets.load_dropdown_lists()

    display(utils.Markdown(data = f"## {task}"))
    display(utils.Markdown(data = "### Select experiment, sample, and instrument"))
    
    display(app_widgets.select_experiment_output)
    
    with app_widgets.select_experiment_output:
        if config["objects"][task]["uses_molecule"]:
            display(app_widgets.experiments_dropdown_boxes, app_widgets.sample_metadata_boxes, app_widgets.instruments_dropdown_boxes, app_widgets.molecule_metadata_boxes)
            app_widgets.molecules_dropdown_boxes.children[0].observe(app_widgets.load_molecule_metadata, names = 'value')
        else:
            display(app_widgets.experiments_dropdown_boxes, app_widgets.sample_metadata_boxes, app_widgets.instruments_dropdown_boxes)
    
    app_widgets.samples_dropdown_boxes.children[0].observe(app_widgets.load_sample_metadata, names = 'value')
    app_widgets.create_button.on_click(app_widgets.create_sample_preparation_action)
    app_widgets.quit_button.on_click(app_widgets.close_notebook)

    display(utils.Markdown(data = "### Properties"))
    display(app_widgets.object_widgets[task])

    display(utils.Markdown(data = "### Support Files"))
    display(app_widgets.support_files_uploader)

    display(utils.Markdown(data = "### Save Sample"))
    app_widgets.property_widgets["$name"].observe(app_widgets.update_text, names = 'value')
    display(app_widgets.sample_out_name_textbox)
    display(app_widgets.bottom_buttons_hbox)
    display(app_widgets.increase_buttons_size)

# Sample Preparation

In [6]:
# Widgets
sample_preparation_dropdown.observe(sample_preparation_widgets, names='value')

# Trigger the callback manually
sample_preparation_widgets({'new': sample_preparation_dropdown.value, 'old': -1})